# Hands-On NLP
## Project : Text Generation and Chatbot for Song Lyrics

<span style="color:magenta">Group Names:</span>

* Petko Petkov
* Manda
* Ilyes SAIS

**1. Generate descriptions  (or the complete instructions that we're going to use for the training of our model) for the song lyrics in the dataset, we should use another more capable model like ChatGPT, Claude, Gemini, etc.** 

## Load Data : 

In [2]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/petkopetkov/spotify-million-song-dataset-descriptions/data/train-00000-of-00001.parquet")

In [3]:
df.head()

,artist,song,link,text,description
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...","romantic, nostalgic, upbeat, sentimental, warm..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...","romantic, gentle, soothing, tender, intimate, ..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,"upbeat, optimistic, reflective, romantic, live..."
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"upbeat, joyful, playful, optimistic, catchy, r..."
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"uplifting, cheerful, energetic, catchy, playfu..."
